In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

2024-09-13 10:29:22.195846: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 10:29:22.198334: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 10:29:22.207371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-13 10:29:22.222274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-13 10:29:22.226799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-13 10:29:22.237266: I tensorflow/core/platform/cpu_feature_gu

In [2]:
import os
os.getcwd()
concat = pd.read_csv('../data/base_dataset_7.csv')
concat.columns

Index(['번지', '본번', '부번', '아파트명', '전용면적', '계약일', '층', '건축년도', 'target',
       'is_test', '주소', 'x', 'y', '계약년', '계약월', '거래취소여부', '거래일건물연식', '구', '동',
       '강남여부', '신축여부', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수'],
      dtype='object')

In [3]:
str_columns = ['본번', '부번', '거래취소여부', '구', '동', '강남여부', '신축여부']

concat[str_columns] = concat[str_columns].astype(str)

# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in concat.columns:
    if pd.api.types.is_numeric_dtype(concat[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'is_test', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수']
범주형 변수: ['번지', '본번', '부번', '아파트명', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부']


In [4]:
# 범주형 변수에 대한 보간
concat[categorical_columns] = concat[categorical_columns].fillna('NULL')

# 연속형 변수에 대한 보간 (선형 보간)
concat[continuous_columns] = concat[continuous_columns].interpolate(method='linear', axis=0)

print(concat.isnull().sum())         # 결측치가 보간된 모습을 확인해봅니다.
print(concat.shape)

번지         0
본번         0
부번         0
아파트명       0
전용면적       0
계약일        0
층          0
건축년도       0
target     0
is_test    0
주소         0
x          0
y          0
계약년        0
계약월        0
거래취소여부     0
거래일건물연식    0
구          0
동          0
강남여부       0
신축여부       0
건축면적       0
연면적        0
대지면적       0
건폐율        0
용적율        0
평균층수       0
dtype: int64
(1128094, 27)


In [5]:
# test와 train 분리
df = concat.query('is_test == 0')  
df_test = concat.query('is_test == 1')  

# 전용면적 375 이상인 것들은 제거
df= df[df['전용면적'] < 375]

# 층 음수 처리
df['층'] = df['층'].apply(lambda x: 1 if x < 0 else x)
df_test['층'] = df_test['층'].apply(lambda x: 1 if x < 0 else x)

# 건축년도 1965년도 이후에 지어진것만 처리
df= df[df['건축년도'] > 1965]

# 거래일 건물 연식 음수 처리
df= df[df['거래일건물연식'] > 0]
concat_select = pd.concat([df, df_test])
concat_select.info()       # 최종 데이터셋은 아래와 같습니다.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1108303 entries, 0 to 1128093
Data columns (total 27 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   번지       1108303 non-null  object 
 1   본번       1108303 non-null  object 
 2   부번       1108303 non-null  object 
 3   아파트명     1108303 non-null  object 
 4   전용면적     1108303 non-null  float64
 5   계약일      1108303 non-null  int64  
 6   층        1108303 non-null  int64  
 7   건축년도     1108303 non-null  int64  
 8   target   1108303 non-null  float64
 9   is_test  1108303 non-null  int64  
 10  주소       1108303 non-null  object 
 11  x        1108303 non-null  float64
 12  y        1108303 non-null  float64
 13  계약년      1108303 non-null  int64  
 14  계약월      1108303 non-null  int64  
 15  거래취소여부   1108303 non-null  object 
 16  거래일건물연식  1108303 non-null  int64  
 17  구        1108303 non-null  object 
 18  동        1108303 non-null  object 
 19  강남여부     1108303 non-null  object 
 20  신축

In [6]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np
import pandas as pd
from tqdm import tqdm

# 하버사인 공식으로 두 지리적 좌표 간의 거리를 계산하는 함수
def haversine_np(lon1, lat1, lon2, lat2):
    R = 6371  # 지구 반경 (킬로미터)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# 최단 거리 계산 함수
def calculate_shortest_distances(concat_coords, facility_coords):
    shortest_distances = []
    for concat_coord in tqdm(concat_coords, total=len(concat_coords)):
        distances = haversine_np(concat_coord[0], concat_coord[1], facility_coords[:, 0], facility_coords[:, 1])
        min_distance = np.min(distances)
        shortest_distances.append(min_distance)
    return shortest_distances

# 데이터 로드
subway_path = pd.read_csv('/Users/minseok/Downloads/xgboost/data/subway_feature.csv')
bus_path = pd.read_csv('/Users/minseok/Downloads/xgboost/data/bus_feature.csv')

# 좌표 배열로 변환
concat_coords = np.array([concat_select['x'], concat_select['y']]).T
subway_coords = np.array([subway_path['경도'], subway_path['위도']]).T
bus_coords = np.array([bus_path['X좌표'], bus_path['Y좌표']]).T

# 지하철과 버스 최단 거리 계산
concat_select['지하철최단거리'] = calculate_shortest_distances(concat_coords, subway_coords)
concat_select['버스최단거리'] = calculate_shortest_distances(concat_coords, bus_coords)

# 거리별 가중치 부여 함수
def assign_weights(distance, thresholds, weights):
    for t, w in zip(thresholds, weights):
        if distance < t:
            return w
    return weights[-1]

# 거리별 가중치 부여
concat_select['지하철최단거리'] = concat_select['지하철최단거리'].apply(lambda x: assign_weights(x, [0.3, 0.65, 1.0], [3, 2, 1, 0]))
concat_select['버스최단거리'] = concat_select['버스최단거리'].apply(lambda x: assign_weights(x, [0.5, 1.0], [2, 1, 0]))

# 결과 확인
concat_select['버스최단거리'].describe()

  0%|          | 0/1108303 [00:00<?, ?it/s]

100%|██████████| 1108303/1108303 [04:06<00:00, 4491.55it/s]


count    1.108303e+06
mean     1.999190e+00
std      3.362743e-02
min      0.000000e+00
25%      2.000000e+00
50%      2.000000e+00
75%      2.000000e+00
max      2.000000e+00
Name: 버스최단거리, dtype: float64

In [7]:
brand_patterns = {
   #1군 하이엔트
   '디에이치':3, '푸르지오써밋': 3, '르엘':3,'오티에르':3, '아크로':3,
   #1군
   '힐스테이트': 2, '푸르지오':2, '롯데캐슬':2, '더샵':2, 'e-편한세상':2,'래미안':2 , '아이파크':2, '자이':2,
   #2군 하이엔드
   '위브더제니스':2,'호반써밋':2,
    #2군
    '위브': 1,'우미린':1,'한화포레나':1, '서희스타힐스':1,'더플래티넘':1, '한라비발디':1,'호반베르디움':1, '데시앙':1, '센트레빌':1,'SKVIEW':1,''
    '하늘채': 1,'스위첸':1
}
# 특정 문자열을 포함하는 경우에 따라 값을 반환하는 함수 정의
def assign_brand_value(apartment_name):
    for pattern, value in brand_patterns.items():
        if pattern in apartment_name:
            return value
    return 0
# '아파트브랜드이름'이라는 새로운 열 생성
concat_select['아파트브랜드이름'] = concat_select['아파트명'].apply(assign_brand_value)
concat_select = concat_select.drop(columns=['아파트명'])
print(concat_select.shape, concat_select.columns)

(1108303, 29) Index(['번지', '본번', '부번', '전용면적', '계약일', '층', '건축년도', 'target', 'is_test', '주소',
       'x', 'y', '계약년', '계약월', '거래취소여부', '거래일건물연식', '구', '동', '강남여부', '신축여부',
       '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리', '버스최단거리',
       '아파트브랜드이름'],
      dtype='object')


In [87]:
# # 금리 데이터 로드 및 처리
# interest = pd.read_csv('/Users/minseok/Downloads/xgboost/data/한국기준금리07-23.csv')

# interest['계약년월'] = interest['날짜'].astype(str).str[:6]

# interest_grouped = interest.groupby('계약년월').agg({'기준금리': 'mean'}).reset_index()

# concat_select['계약년월'] = concat_select['계약년'].astype(str) + concat_select['계약월'].astype(str)

# concat_select = pd.merge(concat_select, interest_grouped, on='계약년월', how='left')



# print(concat_select.shape, concat_select.columns)

In [88]:
# concat_select = concat_select.drop(columns=['계약년월'])
# print(concat_select.shape, concat_select.columns)

In [89]:
# # 필요한 라이브러리 로드
# import numpy as np
# from tqdm import tqdm

# # 금리 데이터 로드 및 처리
# school_path = pd.read_csv('/Users/minseok/Downloads/xgboost/data/청주대학교_지방교육재정연구원_초중등학교위치_20240322.csv', encoding='EUC-KR')

# # 학교별로 데이터를 나눔
# h_school = school_path[school_path['학교급구분'] == '고등학교']
# m_school = school_path[school_path['학교급구분'] == '중학교']
# p_school = school_path[school_path['학교급구분'] == '초등학교']

# # 각 학교의 좌표 추출
# h_school_coords = np.array([h_school['경도'], h_school['위도']]).T
# m_school_coords = np.array([m_school['경도'], m_school['위도']]).T
# p_school_coords = np.array([p_school['경도'], p_school['위도']]).T

# # 최단 거리 계산 함수
# def calculate_shortest_distances(concat_coords, school_coords):
#     shortest_distances = []
#     for concat_coord in tqdm(concat_coords, total=len(concat_coords)):
#         distances = haversine_np(concat_coord[0], concat_coord[1], school_coords[:, 0], school_coords[:, 1])
#         shortest_distances.append(np.min(distances))
#     return shortest_distances

# # 고등학교, 중학교, 초등학교의 최단 거리 계산
# concat_select['고등학교최단거리'] = calculate_shortest_distances(concat_coords, h_school_coords)
# concat_select['중등학교최단거리'] = calculate_shortest_distances(concat_coords, m_school_coords)
# concat_select['초등학교최단거리'] = calculate_shortest_distances(concat_coords, p_school_coords)

# # 거리를 카테고리화하는 함수
# def categorize_distance(distance):
#     if 0 < distance < 0.5:
#         return 3
#     elif 0.5 < distance < 1.0:
#         return 2
#     elif 1.0 < distance < 1.5:
#         return 1
#     else:
#         return 0

# # 각 학교별 최단 거리를 카테고리화
# concat_select['초등학교최단거리'] = concat_select['초등학교최단거리'].apply(categorize_distance)
# concat_select['중등학교최단거리'] = concat_select['중등학교최단거리'].apply(categorize_distance)
# concat_select['고등학교최단거리'] = concat_select['고등학교최단거리'].apply(categorize_distance)

# # 결과 확인
# concat_select.head()

In [8]:
# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_select.query('is_test==0')
dt_test = concat_select.query('is_test==1')

# 결과 출력
print(dt_train.shape, dt_test.shape)

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
dt_test['target'] = 0

(1099031, 29) (9272, 29)
(1099031, 28) (9272, 28)


In [9]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

연속형 변수: ['전용면적', '계약일', '층', '건축년도', 'target', 'x', 'y', '계약년', '계약월', '거래일건물연식', '건축면적', '연면적', '대지면적', '건폐율', '용적율', '평균층수', '지하철최단거리', '버스최단거리', '아파트브랜드이름']
범주형 변수: ['번지', '본번', '부번', '주소', '거래취소여부', '구', '동', '강남여부', '신축여부']


100%|██████████| 9/9 [00:01<00:00,  6.41it/s]


In [10]:
dt_train = pd.get_dummies(dt_train, columns=['구'])
dt_test = pd.get_dummies(dt_test, columns=['구'])

In [11]:
columns_to_delete = ['번지', '부번', '본번']

# # 열 삭제
dt_train = dt_train.drop(columns=columns_to_delete)
dt_test = dt_test.drop(columns=columns_to_delete)

In [12]:
# assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

# # Target과 독립변수들을 분리해줍니다.
# y_train = dt_train['target']
# X_train = dt_train.drop(['target'], axis=1)

# # Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [13]:
# # '계약년' 컬럼의 2023 미만과 2023년 이상 비율을 출력하는 코드
# def print_year_split_ratios(X_train, X_val, y_train, y_val):
#     # 계약년이 2023 미만인 비율과 2023 이상인 비율을 계산하여 출력
#     train_2023_below = (X_train['계약년'] < 2023).mean() * 100
#     train_2023_above = (X_train['계약년'] >= 2023).mean() * 100
#     val_2023_below = (X_val['계약년'] < 2023).mean() * 100
#     val_2023_above = (X_val['계약년'] >= 2023).mean() * 100
    
#     print(f"Training Set - 계약년 2023 미만 비율: {train_2023_below:.2f}%")
#     print(f"Training Set - 계약년 2023 이상 비율: {train_2023_above:.2f}%")
#     print(f"Validation Set - 계약년 2023 미만 비율: {val_2023_below:.2f}%")
#     print(f"Validation Set - 계약년 2023 이상 비율: {val_2023_above:.2f}%")

# # 함수 호출 (데이터 준비 후 실행)
# print_year_split_ratios(X_train, X_val, y_train, y_val)

In [14]:
# #  XGBoost 사용

# import xgboost as xgb

# model = xgb.XGBRegressor(objective='reg:squarederror', 
#                          n_estimators=2000,  
#                          learning_rate=0.1,  
#                          max_depth=10,
#                          random_state=1)

# model.fit(X_train, y_train)

# pred_2 = model.predict(X_val)

# # 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
# print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred_2))}')

In [15]:
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import ParameterGrid
# import joblib
# import os

# # RMSE 계산 함수
# def RMSE(y, y_pred):
#     return np.sqrt(mean_squared_error(y, y_pred))

# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'n_estimators': [9000, 10000, 11000, 12000],  # 더 높은 n_estimators 설정
#     'learning_rate': [0.05, 0.075, 0.1],  # 학습률을 더 세밀하게 설정
#     'max_depth': [5, 6, 7],  # max_depth 범위를 확장하여 조정
#     'min_child_weight': [1, 3],  # 현재 최적값 유지
#     'subsample': [0.8],  # 그대로 유지
#     'colsample_bytree': [0.8],  # 그대로 유지
#     'gamma': [0],  # 그대로 유지
#     'lambda': [1],  # 그대로 유지
#     'alpha': [0]  # 그대로 유지
# }

# # 체크포인트 경로 설정
# checkpoint_path = "grid_search_checkpoint.pkl"

# # 체크포인트에서 학습 재개
# if os.path.exists(checkpoint_path):
#     start_idx, best_params, best_rmse = joblib.load(checkpoint_path)
#     print(f"Checkpoint 로드 완료, {start_idx}번째 파라미터부터 재개.")
# else:
#     start_idx = 0
#     best_params = None
#     best_rmse = float('inf')
#     print("새로운 학습 시작.")

# # XGBRegressor 모델 정의
# xgb_model = xgb.XGBRegressor(
#     objective='reg:squarederror',
#     random_state=2024
# )

# # 파라미터 그리드 순회
# for idx, params in enumerate(ParameterGrid(param_grid)):
#     if idx < start_idx:
#         continue  # 이전에 완료된 파라미터 조합은 건너뜀
    
#     # 모델 학습
#     print(f"{idx + 1}번째 파라미터 조합 학습 중: {params}")
#     xgb_model.set_params(**params)
#     xgb_model.fit(X_train, y_train)

#     # 검증 세트에서 예측
#     pred_val = xgb_model.predict(X_val)
#     val_rmse = RMSE(y_val, pred_val)
#     print(f"Validation RMSE: {val_rmse:.4f}")

#     # 최적 파라미터 갱신
#     if val_rmse < best_rmse:
#         best_rmse = val_rmse
#         best_params = params

#     # 진행 상황을 체크포인트에 저장
#     joblib.dump((idx + 1, best_params, best_rmse), checkpoint_path)
#     print(f"Checkpoint 저장됨: {checkpoint_path}")

# # 최적 파라미터 및 성능 출력
# print(f"최적 파라미터: {best_params}, Best RMSE: {best_rmse:.4f}")

# # 최적 모델로 학습 후 모델 저장
# best_xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)
# best_xgb_model.fit(X_train, y_train)
# best_xgb_model.save_model(f"xgb_best_model_rmse_{best_rmse:.4f}.model")

# # 체크포인트 삭제
# if os.path.exists(checkpoint_path):
#     os.remove(checkpoint_path)
#     print("Checkpoint 파일 삭제 완료.")

In [51]:
import numpy as np

# y_train, y_val에서 상위 10%에 해당하는 임계값을 계산
top_10_percent_train_threshold = np.percentile(y_train, 90)  # 상위 10%의 임계값 계산
top_10_percent_val_threshold = np.percentile(y_val, 90)      # 검증 데이터 상위 10% 임계값 계산

# y_train에서 상위 10%에 해당하는 값 필터링 후 정렬 (내림차순)
top_10_percent_train = np.sort(y_train[y_train >= top_10_percent_train_threshold])[::-1]
# y_val에서 상위 10%에 해당하는 값 필터링 후 정렬 (내림차순)
top_10_percent_val = np.sort(y_val[y_val >= top_10_percent_val_threshold])[::-1]

# 상위 10%에 해당하는 값의 개수 출력
print(f"y_train 상위 10% 값의 개수: {len(top_10_percent_train)}개")
print(f"y_val 상위 10% 값의 개수: {len(top_10_percent_val)}개")

# 상위 10%에 해당하는 값 출력 (내림차순 정렬)
print(f"y_train 상위 10% 값 (내림차순):\n{top_10_percent_train}")
print(f"y_val 상위 10% 값 (내림차순):\n{top_10_percent_val}")

y_train 상위 10% 값의 개수: 88049개
y_val 상위 10% 값의 개수: 22242개
y_train 상위 10% 값 (내림차순):
[1450000. 1350000. 1300000. ...  109000.  109000.  109000.]
y_val 상위 10% 값 (내림차순):
[1100000. 1080000.  970000. ...  108000.  108000.  108000.]


In [53]:
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# import joblib
# import os

# # RMSE 계산 함수
# def RMSE(y, y_pred):
#     return np.sqrt(mean_squared_error(y, y_pred))

# # 가중치 부여 함수
# def calculate_sample_weights(target, threshold=107000, high_weight=3, low_weight=1):
#     """
#     집값이 threshold 이상이면 high_weight, 이하이면 low_weight 가중치를 부여
#     """
#     return np.where(target >= threshold, high_weight, low_weight)

# # 최적 파라미터 설정
# best_params = {
#     'n_estimators': 12000,  
#     'learning_rate': 0.05,  
#     'max_depth': 7,  
#     'min_child_weight': 1,  
#     'subsample': 0.8,  
#     'colsample_bytree': 0.8,  
#     'gamma': 0,  
#     'lambda': 1,  
#     'alpha': 0  
# }

# # 체크포인트 경로 설정
# checkpoint_path = "best_model_checkpoint.pkl"

# # 체크포인트에서 학습 재개
# if os.path.exists(checkpoint_path):
#     print("Checkpoint 로드 완료.")
#     best_xgb_model = joblib.load(checkpoint_path)
# else:
#     print("새로운 학습 시작.")

# # XGBRegressor 모델 정의
# best_xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)

# # 가중치 계산 (y_train에 대해 가중치를 부여)
# sample_weights = calculate_sample_weights(y_train, threshold=1000000, high_weight=3, low_weight=1)

# # 최적 모델 학습
# print(f"최적 파라미터로 XGBoost 학습 시작: {best_params}")
# best_xgb_model.fit(X_train, y_train, sample_weight=sample_weights)

# # 검증 세트에서 예측
# pred_val = best_xgb_model.predict(X_val)
# val_rmse = RMSE(y_val, pred_val)
# print(f"Validation RMSE: {val_rmse:.4f}")

# # 최종 모델 저장
# best_xgb_model.save_model(f"xgb_best_model_rmse_{val_rmse:.4f}.model")
# joblib.dump(best_xgb_model, checkpoint_path)  # 학습된 모델을 저장

# # 체크포인트 삭제 (선택사항)
# if os.path.exists(checkpoint_path):
#     os.remove(checkpoint_path)
#     print("Checkpoint 파일 삭제 완료.")

새로운 학습 시작.
최적 파라미터로 XGBoost 학습 시작: {'n_estimators': 12000, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'alpha': 0}
Validation RMSE: 5421.1373
Checkpoint 파일 삭제 완료.


In [16]:
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# import joblib
# import os

# # RMSE 계산 함수
# def RMSE(y, y_pred):
#     return np.sqrt(mean_squared_error(y, y_pred))

# # 최적 파라미터 설정
# best_params = {
#     'n_estimators': 12000,  
#     'learning_rate': 0.05,  
#     'max_depth': 7,  
#     'min_child_weight': 3,  
#     'subsample': 0.8,  
#     'colsample_bytree': 0.8,  
#     'gamma': 0,  
#     'lambda': 1,  
#     'alpha': 0  
# }

# # RMSE 결과 출력
# print("RMSE test: 5435.876848555208")
# print(f"최적 파라미터: {best_params}")
# # 체크포인트 경로 설정
# checkpoint_path = "best_model_checkpoint.pkl"

# # 체크포인트에서 학습 재개
# if os.path.exists(checkpoint_path):
#     print("Checkpoint 로드 완료.")
#     best_xgb_model = joblib.load(checkpoint_path)
# else:
#     print("새로운 학습 시작.")

# # 로그 변환된 y_train과 y_val 사용
# y_train_log = np.log(y_train)  # 타겟 변수를 로그 변환
# y_val_log = np.log(y_val)  # 검증용 타겟 변수도 로그 변환

# # XGBRegressor 모델 정의
# best_xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)

# # 최적 모델 학습 (로그 변환된 타겟으로 학습)
# print(f"최적 파라미터로 XGBoost 학습 시작: {best_params}")
# best_xgb_model.fit(X_train, y_train_log)

# # 검증 세트에서 예측 (로그 변환된 값으로 예측)
# pred_val_log = best_xgb_model.predict(X_val)

# # 로그 변환된 값과 비교하여 RMSE 계산
# val_rmse_log = RMSE(y_val_log, pred_val_log)  # 로그 변환된 값에서 RMSE 계산
# print(f"Validation RMSE (로그 변환된 값): {val_rmse_log:.4f}")

# # 원래 값으로 변환한 후, 실제 RMSE 확인
# pred_val = np.exp(pred_val_log)  # 예측 값 지수 변환
# val_rmse = RMSE(y_val, pred_val)  # 원래 값과 비교하여 RMSE 계산
# print(f"Validation RMSE (지수 변환 후 실제 값): {val_rmse:.4f}")

# # 최종 모델 저장
# best_xgb_model.save_model(f"xgb_best_model_rmse_{val_rmse:.4f}.model")
# joblib.dump(best_xgb_model, checkpoint_path)  # 학습된 모델을 저장

# # 체크포인트 삭제 (선택사항)
# if os.path.exists(checkpoint_path):
#     os.remove(checkpoint_path)
#     print("Checkpoint 파일 삭제 완료.")

In [18]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# RMSE 계산 함수
def RMSE(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# 로그 변환된 RMSE를 계산하기 위한 함수
def calculate_rmse_with_kfold(X_train, y_train, best_params, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=2024)
    fold_rmse = []
    
    # K-fold 교차 검증 시작
    for fold, (train_index, val_index) in enumerate(kf.split(X_train), start=1):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # 로그 변환
        y_train_log_fold = np.log(y_train_fold)
        y_val_log_fold = np.log(y_val_fold)
        
        # XGBRegressor 모델 정의
        xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)
        
        # 모델 학습
        print(f"Fold {fold}: 학습 시작")
        xgb_model.fit(X_train_fold, y_train_log_fold)
        
        # 검증 세트에서 예측
        pred_val_log = xgb_model.predict(X_val_fold)
        
        # 원래 값으로 변환한 후, 실제 RMSE 확인
        pred_val = np.exp(pred_val_log)
        val_rmse = RMSE(y_val_fold, pred_val)
        
        # RMSE 결과 저장
        fold_rmse.append(val_rmse)
        print(f"Fold {fold} RMSE: {val_rmse:.4f}")
    
    # 모든 fold에 대한 평균 RMSE 출력
    average_rmse = np.mean(fold_rmse)
    print(f"Average RMSE across {n_splits} folds: {average_rmse:.4f}")
    
    return average_rmse

# 최적 파라미터 설정
best_params = {
    'n_estimators': 12000,  
    'learning_rate': 0.05,  
    'max_depth': 7,  
    'min_child_weight': 3,  
    'subsample': 0.8,  
    'colsample_bytree': 0.8,  
    'gamma': 0,  
    'lambda': 1,  
    'alpha': 0  
}

# 데이터 로드 및 Target과 독립변수 분리
y_train = dt_train['target']
X_train = dt_train.drop(['target'], axis=1)

# K-fold 적용하여 RMSE 계산
average_rmse = calculate_rmse_with_kfold(X_train, y_train, best_params, n_splits=5)

# 최종 학습 및 모델 저장
checkpoint_path = "best_model_checkpoint.pkl"

# 로그 변환된 y_train 사용
y_train_log = np.log(y_train)

# XGBRegressor 모델 정의 (최종 모델 학습)
best_xgb_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', random_state=2024)

# 최적 모델 학습 (로그 변환된 타겟으로 학습)
print(f"최적 파라미터로 XGBoost 학습 시작: {best_params}")
best_xgb_model.fit(X_train, y_train_log)

# 최종 모델 저장
val_rmse = average_rmse  # K-fold의 평균 RMSE로 저장 파일명에 사용
best_xgb_model.save_model(f"xgb_best_model_rmse_{val_rmse:.4f}.model")
joblib.dump(best_xgb_model, checkpoint_path)  # 학습된 모델을 저장

# 체크포인트 삭제 (선택사항)
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)
    print("Checkpoint 파일 삭제 완료.")

Fold 1: 학습 시작
Fold 1 RMSE: 5394.4457
Fold 2: 학습 시작
Fold 2 RMSE: 5635.8242
Fold 3: 학습 시작
Fold 3 RMSE: 5649.0600
Fold 4: 학습 시작
Fold 4 RMSE: 5532.8477
Fold 5: 학습 시작
Fold 5 RMSE: 5788.1907
Average RMSE across 5 folds: 5600.0737
최적 파라미터로 XGBoost 학습 시작: {'n_estimators': 12000, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_weight': 3, 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'lambda': 1, 'alpha': 0}
Checkpoint 파일 삭제 완료.


In [19]:
# from sklearn import metrics

# # 예측 수행
# pred_2 = best_xgb_model.predict(X_val)

# # RMSE 계산
# rmse_result = np.sqrt(metrics.mean_squared_error(y_val, pred_2))

# # 결과 출력 및 파일 저장
# print(f'RMSE test: {rmse_result}')

# # 결과를 파일에 저장
# with open("rmse_result.txt", "w") as f:
#     f.write(f'RMSE test: {rmse_result}\n')
#     f.write(f"최적 파라미터: {best_params}, Best RMSE: {best_rmse:.4f}")

In [22]:
from sklearn import metrics

# 예측 수행
pred_2 = best_xgb_model.predict(X_train)
pred_val = np.exp(pred_2)  # 예측 값 지수 변환
rmse_result = RMSE(y_train, pred_val)  # 원래 값과 비교하여 RMSE 계산

# # RMSE 계산
# rmse_result = np.sqrt(metrics.mean_squared_error(y_val, pred_2))

# 결과 출력 및 파일 저장
print(f'RMSE test: {rmse_result}')

# 결과를 파일에 저장
with open("rmse_result.txt", "w") as f:
    f.write(f'RMSE test: {rmse_result}\n')
    f.write(f"최적 파라미터: {best_params}")

RMSE test: 3572.750454907125


In [29]:
# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(best_xgb_model, f)

In [30]:
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [31]:
# X_test = dt_test.drop(['target'], axis=1)

# real_test_pred = model.predict(X_test)

In [32]:
X_test = dt_test.drop(['target'], axis=1)

real_test_pred = model.predict(X_test)
real_pred_val = np.exp(real_test_pred)

In [33]:
# # 앞서 예측한 예측값들을 저장합니다.
# preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
# preds_df.to_csv('output.csv', index=False)

In [34]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_pred_val.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)